# SEQP Personalized Signal Generator
This is a Jupyter notebook designed to create a personalized version of the SEQP test signal. 

In [1]:
call = 'KD8OXT'
maidenhead = 'EN91em'

### 0) Housekeeping: Import packages and set audio parameters

In [2]:
fs = 48000;                   # samples per second
dt = 1/fs;                    # seconds per sample
wpm = 20

## 1) Generate pseudorandom signal from user's callsign and grid square.
Let's generate two seconds of static - one from the callsign, one from the grid square. We'll stick them together.

In [3]:
import random
hash(call)

-5021921347211812440

In [4]:
random.seed(hash(call))
random.SystemRandom(call)


## 2) Generate Morse signal from user's callsign. 
This signal is at 20 WPM. All callsigns must fit in the same time block... and should be received by RBN.

In [6]:
msg = 'TEST de ' + call + "   " + maidenhead+ "   "
msg = msg * 3
print(msg)

TEST de KD8OXT   EN91em   TEST de KD8OXT   EN91em   TEST de KD8OXT   EN91em   


Let's convert the signal to an on-off signal:

In [14]:
from funny_morse import *

# import morse
# from morse import *
# vars(morse)
# msg_morse = morse.stringToMorse('test')                      # convert from string to CW characters

msg_morse = morse.string_to_morse('hi there')

sps = 400
wpm = 20
# fs = 2800

# msg_array = morse.morseToBoolArr(msg_morse, sps, wpm)    # convert from CW characters to Boolean array

# print(msg_array)

# msg_cw = msg_array.astype(int)
# len(msg_cw)

ImportError: this platform is not supported: ('failed to acquire X connection: Can\'t connect to display "localhost:0.0": [Errno 111] Connection refused', DisplayConnectionError('localhost:0.0', '[Errno 111] Connection refused'))

Try one of the following resolutions:

 * Please make sure that you have an X server running, and that the DISPLAY environment variable is set correctly

In [8]:
funny_morse --wpm 15 --fs 15 -p hello

SyntaxError: invalid syntax (983197306.py, line 1)

In [13]:
# from morse import *
# vars(morse)

# window("show", wpm=5, fs=None)

We have to make sure this fits into a standard time frame, so let's prallocate an array of desired length and then add it in:

## 3) Concatenate PRN, CW, and radar chirps

Now we add in the lightsaber noises. Er, chirps. 

Chirps from [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.7659302.svg)](https://doi.org/10.5281/zenodo.7659302).

In [ ]:
# 'https://zenodo.org/record/7659302/files/seqp-test.csv?download=1'
import pandas as pd
data = pd.read_csv('https://zenodo.org/record/7659302/files/seqp-test.csv?download=1')
data

## 4) Save File
Let's save the resulting .CSV and .WAV files with mnemonic filenames that include callsign and grid square.